## Sequence Transformer

**You start with the sequence ABABAECCEC, or in general any sequence made from A,B, C, and E.** 

**You can transform this sequence using the following equalities: AC = E,AB = BC, BB = E, CC = E, and E x = x for any x.** 

**Your goal is to produce the sequence E.**

**For example, ABBC can be transformed into AEC, and then AC, and then E.**

In [1]:
import ipywidgets as widgets
from IPython.display import clear_output

In [2]:
input = widgets.Text(description="Sequence")

In [3]:
class Priority_Queue:
    
    def __init__(self):
        self.queue = []
    
    def enqueue(self, node):
        self.queue.append(node)
        
    def dequeue(self):
        if not self.empty():
            max_priority_idx = 0
            for i in range(len(self.queue)): 
                if self.queue[i].priority > self.queue[max_priority_idx].priority : 
                    max_priority_idx = i 
            item = self.queue[max_priority_idx] 
            del self.queue[max_priority_idx] 
            return item 
        else:
            raise Exception("Priority_Queue Empty!")
            
    def empty(self):
        return len(self.queue) == 0
    
    def contains_state(self, state):
        return any(node.state == state for node in self.queue)
    
    def __str__(self):
        result = f"Number of items in queue = {len(self.queue)}\n"
        for item in self.queue:
            result += f"{item}\n"
        return result    

In [4]:
class Node:
    def __init__(self, state, parent, action, depth, priority=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
        self.priority = priority
    
    def repeated_state(self):
        if self.parent == None or self.parent.parent == None: 
            return False
        if self.parent.parent.state.equals(self.state): 
            return True
        return False
    
    def __str__(self):
        result = f"{self.state}"
        result += f" {self.depth}"
        result += f" {self.priority}"
        if self.parent != None:
            result += f" {self.parent.state}"
            result += f" {self.action}"
        return result  

In [5]:
class Search:
    
    # A* Search
    
    def __init__(self, start_state, goal_state):

        self.start_state = start_state
        self.goal_state = goal_state
            
    def heuristic(self, node):      
        
        def get_count(sub):
            c = seq.count(sub)
            if c != -1: return c
            return 0
        
        seq = node[0].sequence
        
        #Admissible Heuristic
        return get_count("AC") + get_count("BB") + get_count("CC")
    
    def find_solution(self):
        
        # Initialize fringe
        start_node = Node(self.start_state, None, None, 0)
        fringe = Priority_Queue()
        fringe.enqueue(start_node)
        
        # Initialize an empty explored set
        self.explored = set()
        
        while not fringe.empty():
            
            # Choose a node from the fringe
            current_node = fringe.dequeue()
            
            if not current_node.state.legal():
                return None
            
            # If the chosen node is the goal, then we have a solution
            if self.goal_state.equals(current_node.state):
                #print(f"No. of nodes explored = {len(self.explored)}")
                return current_node

            # Mark node as explored
            self.explored.add(current_node.state)
            
            # Add possible successors to the fringe
            successors = current_node.state.find_successors()
            for successor in successors:   
                if not fringe.contains_state(successor[0]) and successor[0] not in self.explored:
                    n = Node(successor[0],#state
                             current_node,
                             successor[1],#action
                             current_node.depth+1, current_node.depth + 1 + self.heuristic(successor))
                    if n.repeated_state():
                        del(n)
                    else:
                        fringe.enqueue(n)
            #print(fringe)
        return None
    
    def build_path(self, node):
        result = []
        while node:
            result.insert(0, node)
            node = node.parent
        return result
    
    def show_path(self, node):
        path = self.build_path(node)        
        for current_node in path:
            if current_node.action:
                print(current_node.action)
            print(current_node.state)
        print(f"Number of steps = {current_node.depth}")
    
    def solve(self):
        solution = self.find_solution()
        if solution == None:
            print("Search failed")
        else:
            print("Search completed successfully!")
            self.show_path(solution)

In [6]:
def replace_nth(s, old, new, n):
    count = 0

    find = s.find(old)
    if find != -1: count += 1
        
    while count != n and find != -1:
        find = s.find(old, find + 1)
        if find != -1: count += 1
            
    if count == n:
        return s[:find] + new + s[find + len(old):]

In [7]:
class Sequence_State:

    def __init__(self, sequence):
        self.sequence = sequence
       
    def __str__(self):    
        return self.sequence
    
    def equals(self, state):
        return (self.sequence == state.sequence)
    
    def legal(self):
        allowed = {'A', 'B', 'C', 'E'}
        current = set(self.sequence)
        if current.issubset(allowed):
            return True
        return False
    
    def apply_helper(self, successors, old, new, op):
        number_of_occurrences = self.sequence.count(old)
        count = 1
        while(count <= number_of_occurrences) :
            s = Sequence_State(replace_nth(self.sequence, old, new, count))
            if s.sequence != None: successors.append((s, op))
            count += 1
        
    
    # There are five possible transformations:
    # 1. AC = E
    def apply_AC_E(self, successors):
        self.apply_helper(successors, "AC", "E", "AC = E")
    
    # 2. AB = BC
    def apply_AB_BC(self, successors):
        self.apply_helper(successors, "AB", "BC", "AB = BC")
    
    # 3. BB = E
    def apply_BB_E(self, successors):
        self.apply_helper(successors, "BB", "E", "BB = E")
    
    # 4. CC = E
    def apply_CC_E(self, successors):
        self.apply_helper(successors, "CC", "E", "CC = E")
    
    # 5. Ex = x for any x
    def apply_Ex_x(self, successors):
        self.apply_helper(successors, "EA", "A", "Ex = x (EA = A)")
        self.apply_helper(successors, "EB", "B", "Ex = x (EB = B)")
        self.apply_helper(successors, "EC", "C", "Ex = x (EC = C)")
        self.apply_helper(successors, "EE", "E", "Ex = x (EE = E)")
     
    def find_successors(self):
        successors = []
        self.apply_AC_E(successors)
        self.apply_AB_BC(successors)
        self.apply_BB_E(successors)
        self.apply_CC_E(successors)
        self.apply_Ex_x(successors)
        return successors

In [8]:
button_search = widgets.Button(description="Search")

def on_button_search_clicked(b):
        with output:
            clear_output(True)
            s = Search(Sequence_State(input.value), Sequence_State("E"))
            s.solve()

button_search.on_click(on_button_search_clicked)

In [9]:
# Test Cases
# ABBC
# ABABAECCEC
# ABBCX
# ABABAECCECABBC
# ABABAECCECABABAECCEC
# ABABAECCECABABAECCECABABAECCECABABAECCEC

In [10]:
display(input)

Text(value='', description='Sequence')

In [11]:
display(button_search)

Button(description='Search', style=ButtonStyle())

In [12]:
output = widgets.Output()
display(output)

Output()